# 1.
I think that using introspection to observe how human cognative processes work in order to model them is a BAD way for researchers to gain insight. Human minds are so complicated that i dont think it would be possible for one person to be able to simultaneously be able to understand/handle the immense amount of factors involved in decision making. A person who was trying to make a robot would have to somehow emulate emotions, physical sensation, memories, cognative and logical ability, and all the other senses that a person experiences all of which are factors in how a person makes a decision.

# 2. 
My state representation is an array of integers 0...n where n is the number of nodes in the graph.
My action representation is an array of all possible integer pairs of swappable indices. Each pair shows up in the action list only once.
The hill-climbing search works far better than the simulated annealing algorithm because the hill-climbing search finds a local maximum and immediately stops whereas the simulated annealing will continue to search even after it may have found a better solution.


In [15]:
from search import Problem, hill_climbing, simulated_annealing, exp_schedule, genetic_search
import math
import itertools
import time 

class TSP(Problem):

    def __init__(self, initial, num_nodes, distance_matrix):
        self.initial = initial[:]
        self.num_nodes = num_nodes
        self.distance_matrix = distance_matrix[:]
        self.state = initial[:]

    def get_state(self):
        """Returns the final state of the algorithm"""
        return self.state

    def actions(self, state):
        """Returns an array of unique pairs of indices that can be swapped"""
        actions = []
        for x in range(self.num_nodes - 1):
            for y in range(x + 1, self.num_nodes):
                if(x != y):
                    actions.append([x,y])
        return actions

    def result(self, state, action):
        """Return the state that results from executing the given
        action in the given state. The action must be one of
        self.actions(state)."""
        result = state[:]
        temp = result[action[0]]
        result[action[0]] = result[action[1]]
        result[action[1]] = temp

        return result

    def value(self, state):
        """For optimization problems, each state has a value.  Hill-climbing
        and related algorithms try to maximize this value."""
        result = 0
        for x in range(self.num_nodes - 1):
            result += self.distance_matrix[state[x]][state[x+1]]
        result += self.distance_matrix[state[self.num_nodes -1 ]][state[0]]
        self.state = state
        return -result

if __name__ == '__main__':

    # Create 4x4 distance map for the 4 cities
    num_cities = 4
    tsp_city_distances = [
        [0,2,3,4],
        [2,0,8,4],
        [3,8,0,4],
        [4,4,4,0]
    ]

    # Set random inital state
    initial_state = [ 0, 1, 2, 3 ]

    p = TSP(initial_state, num_cities, tsp_city_distances)

    # Solve the problem using hill-climbing.
    time1 = time.time()
    hill_solution = hill_climbing(p)
    time2 = time.time()

    # Solve the problem using simulated annealing.
    time3 = time.time()
    annealing_solution = simulated_annealing(
        p,
        exp_schedule(k=20, lam=0.005, limit=1000)
    )
    time4 = time.time()

    # Print solutions
    print("Hill climbing solution: ", p.get_state(), "\nWith a distance of: ", -p.value(hill_solution), "\nIn a time of :", time2 - time1) 

    print("Simulated Annealing solution: ", p.get_state(), "\nWith a distance of: ", -p.value(annealing_solution), "\nIn a time of :", time4 - time3)

Hill climbing solution:  [3, 2, 0, 1] 
With a distance of:  13 
In a time of : 6.699562072753906e-05
Simulated Annealing solution:  [1, 0, 2, 3] 
With a distance of:  13 
In a time of : 0.01299142837524414


# 3


In [32]:
from csp import CSP, min_conflicts, backtracking_search, AC3, parse_neighbors
from search import depth_first_graph_search
def CourseSchedule():
    """Return an instance of the Course Scheduling Puzzle."""

    # create list of variables
    Courses = '108 112 214 220 232 262'.split()
    FacultyMembers = 'adams plantinga norman vanderlinden'.split()
    TimeSlots = 'mwf9 mwf10 mwf11 mwf12'.split()
    Classrooms = 'nh253 sb382'.split()
    variables = Courses
    # create an array of every possible teacher-time-room combination so that we can set the domain for each variable/course
    domain = {}
    i = 0
    ttrcombination = [0] * 32
    for x in range(4):
        for y in range(4):
            for z in range(2):
                ttrcombination[i] = [FacultyMembers[x], TimeSlots[y], Classrooms[z]]
                i += 1
    for course in Courses:
        domain[course] = ttrcombination

    # set up neighbors for the variables (the courses)
    neighbors = parse_neighbors("""
                108: 112 212 214 220 232 262;
                112: 212 214 220 232 262;
                212: 214 220 232 262;
                214: 220 232 262; 
                220: 232 262;
                232: 262; 
                262:
                """, variables)

    def zebra_constraint(A, a, B, b, recurse=0):
        result = True
        # same course cannot be taught multiple times
        if A == B:
            result = False
        # teacher cant be in two places at once
        elif a[0] == b[0] and a[1] == b[1]:
            return False
        # classes cant be taught in same room at same time
        elif a[1] == b[1] and a[2] == b[2]:
            return False
        # courses can only be taught by the teachers who teach them
        elif A == '108' and a[0] != 'norman':
            result = False
        elif A == '112' and a[0] != 'adams':
            result = False
        elif A == '212' and a[0] != 'plantinga':
            result = False
        elif A == '214' and a[0] != 'adams':
            result = False
        elif A == '220' and a[0] != 'norman':
            result = False
        elif A == '232' and a[0] != 'norman':
            result = False
        elif A == '262' and a[0] != 'vanderlinden':
            result = False
        elif B == '108' and b[0] != 'norman':
            result = False
        elif B == '112' and b[0] != 'adams':
            result = False
        elif B == '212' and b[0] != 'plantinga':
            result = False
        elif B == '214' and b[0] != 'adams':
            result = False
        elif B == '220' and b[0] != 'norman':
            result = False
        elif B == '232' and b[0] != 'norman':
            result = False
        elif B == '262' and b[0] != 'vanderlinden':
            result = False
        return result

    return CSP(variables, domain, neighbors, zebra_constraint)

if __name__ == '__main__':
    solution = min_conflicts(CourseSchedule())
    print(solution)

{'108': ['norman', 'mwf11', 'sb382'], '112': ['adams', 'mwf11', 'nh253'], '214': ['adams', 'mwf12', 'sb382'], '220': ['norman', 'mwf9', 'sb382'], '232': ['norman', 'mwf10', 'nh253'], '262': ['vanderlinden', 'mwf9', 'nh253']}
